# Word Prediction using Markov Model

This notebook makes use of Markov model for word prediction. Specifically 2nd order Markov model is deployed here for next word prediction. As an example of the Markov chain, an attempt is made to generate a new song lyrics from a bunch of Eminem song lyrics.

In [0]:
# Preamble
import string
import numpy as np

In [0]:
# Path of the text file containing the training data
training_data_file = 'eminem_songs_lyrics.txt'

## Training

### Helper functions

In [0]:
def remove_punctuation(sentence):
    return sentence.translate(str.maketrans('','', string.punctuation))

In [0]:
def add2dict(dictionary, key, value):
    if key not in dictionary:
        dictionary[key] = []
    dictionary[key].append(value)

In [0]:
def list2probabilitydict(given_list):
    probability_dict = {}
    given_list_length = len(given_list)
    for item in given_list:
        probability_dict[item] = probability_dict.get(item, 0) + 1
    for key, value in probability_dict.items():
        probability_dict[key] = value / given_list_length
    return probability_dict

In [0]:
initial_word = {}
second_word = {}
transitions = {}

### Training function

In [0]:
# Trains a Markov model based on the data in training_data_file
def train_markov_model():
    for line in open(training_data_file, encoding="utf8"):
        tokens = remove_punctuation(line.rstrip().lower()).split()
        tokens_length = len(tokens)
        for i in range(tokens_length):
            token = tokens[i]
            if i == 0:
                initial_word[token] = initial_word.get(token, 0) + 1
            else:
                prev_token = tokens[i - 1]
                if i == tokens_length - 1:
                    add2dict(transitions, (prev_token, token), 'END')
                if i == 1:
                    add2dict(second_word, prev_token, token)
                else:
                    prev_prev_token = tokens[i - 2]
                    add2dict(transitions, (prev_prev_token, prev_token), token)
    
    # Normalize the distributions
    initial_word_total = sum(initial_word.values())
    for key, value in initial_word.items():
        initial_word[key] = value / initial_word_total
        
    for prev_word, next_word_list in second_word.items():
        second_word[prev_word] = list2probabilitydict(next_word_list)
        
    for word_pair, next_word_list in transitions.items():
        transitions[word_pair] = list2probabilitydict(next_word_list)
    
    print('Training successful.')

In [0]:
train_markov_model()

Training successful.


## Testing

### Helper functions

In [0]:
def sample_word(dictionary):
    p0 = np.random.random()
    cumulative = 0
    for key, value in dictionary.items():
        cumulative += value
        if p0 < cumulative:
            return key
    assert(False)

### Test functions

In [0]:
number_of_sentences = 10

In [0]:
# Function to generate sample text
print(initial_word)
def generate():
    for i in range(number_of_sentences):
        sentence = []
        # Initial word
        word0 = sample_word(initial_word)
        sentence.append(word0)
        # Second word
        word1 = sample_word(second_word[word0])
        sentence.append(word1)
        # Subsequent words untill END
        while True:
            word2 = sample_word(transitions[(word0, word1)])
            if word2 == 'END':
                break
            sentence.append(word2)
            word0 = word1
            word1 = word2
        print(' '.join(sentence))

{'im': 0.023529411764705882, 'lifes': 0.002352941176470588, 'best': 0.002352941176470588, 'big': 0.002352941176470588, 'this': 0.018823529411764704, 'yella': 0.002352941176470588, 'immortality': 0.002352941176470588, 'no': 0.002352941176470588, 'too': 0.002352941176470588, 'basically': 0.002352941176470588, 'soon': 0.002352941176470588, 'make': 0.004705882352941176, 'bumping': 0.002352941176470588, 'uh': 0.002352941176470588, 'detergent': 0.002352941176470588, 'six': 0.004705882352941176, 'feet': 0.002352941176470588, 'why': 0.004705882352941176, 'after': 0.002352941176470588, 'full': 0.002352941176470588, 'lonely': 0.002352941176470588, 'easy': 0.002352941176470588, 'river': 0.004705882352941176, 'like': 0.004705882352941176, 'think': 0.002352941176470588, 'maybe': 0.002352941176470588, 'and': 0.05647058823529412, 'when': 0.007058823529411765, 'stay': 0.002352941176470588, 'til': 0.002352941176470588, 'in': 0.002352941176470588, 'everybody': 0.004705882352941176, 'feel': 0.00470588235

### Testing arena

In [0]:
generate()

until the roof comes off till the lights go out
while he play piano
fcuk can i say it was ironic
trying to hold onto it
well little one i dont want to admit to something
feel weak you feel em
i dont want to admit to something
back to his mobile home thats when its
feel my wrath of attack
its curtains im inadvertently hurtin you
